In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb 

In [2]:
data = pd.read_csv('consumption_interpolated_updated.csv')

In [ ]:
# display auto-correlation graph 
from pandas.plotting import autocorrelation_plot as auto_p
plt.figure(figsize=(20,5))
f = auto_p(data['consumption'])
# f.set_xlim([1,500])

In [3]:
# reformat time-string 
data['date'] = data['Unnamed: 0'].apply(lambda x: x.split(' ')[0])
data['time'] = data['Unnamed: 0'].apply(lambda x: x.split(' ')[1])
data = data.drop(['Unnamed: 0'],axis=1)

In [4]:
# retrive date informations from 'date'
data['date'] = data['date'].astype('datetime64[D]')
data['week'] = data['date'].dt.week
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.dayofyear

In [5]:
def Season(x):
    if 1<=x<=3: 
        return 4
    elif 4<=x<=6:
        return 1
    elif 7<=x<=9:
        return 2
    else:
        return 3
    
# retrive hour information 
data['hour'] = data['time'].apply(lambda x: float(x.split(':')[0])) + data['time'].apply(lambda x: float(x.split(':')[1])/60)
data['minuteOfDay'] = data['time'].apply(lambda x: float(x.split(':')[1])) + 60*data['hour']
data['working?'] = data['hour'].apply(lambda x: x>6 and x<20)
data['season'] = data['month'].apply(Season)
                                                                            
data = data.drop(['time','date','hour'],axis=1)

In [6]:
# determine the columns' type 
for col in data.columns:
    data[col] = data[col].astype('float')

In [7]:
# delete abnormal time series 
data = data.loc[11000:]

In [11]:
from math import sqrt
error = test['consumption'] - test['consumption'].shift(96)
print(error)
print(error.shape)
RMSE = sqrt((error*error).sum()/(error.shape[0]-96))
print(RMSE)

48480         NaN
48481         NaN
48482         NaN
48483         NaN
48484         NaN
           ...   
64539    0.147626
64540    0.147757
64541    0.049817
64542    0.013954
64543    0.035211
Name: consumption, Length: 16064, dtype: float64
(16064,)
0.1978300266442752


In [8]:
# train, test split 
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size=0.3,shuffle=False)

In [ ]:
origin = test
origin['consumption'].to_csv('Original_y0.csv',index=False)

In [ ]:
# assign data with train and test to continue! 

In [ ]:
# from scipy.signal import savgol_filter

# smoothed_2dg = savgol_filter(train['consumption'], window_length = 5, polyorder = 2)

# smoothed_1dg = savgol_filter(train['consumption'], window_length = 5, polyorder = 1)

# data['smoothed_2dg'] = smoothed_2dg
# data['smoothed_1dg'] = smoothed_1dg

# # f,axes = plt.subplots(3,1,figsize=(75,15))
# # a = sb.lineplot(x=data.index,y='smoothed_2dg',data=data,ax=axes[0])
# # a.set_xlim([11000,11500])
# # b = sb.lineplot(x=data.index,y='smoothed_1dg',data=data,ax=axes[1])
# # b.set_xlim([11000,11500])
# # c = sb.lineplot(x=data.index,y='consumption',data=data,ax=axes[2])
# # c.set_xlim([11000,11500])


In [ ]:
# data['consumption'] = data['smoothed_2dg']
# data.drop(['smoothed_2dg','smoothed_1dg'],axis=1,inplace=True)
# data

In [ ]:
# data['difference'] = data['Consumption'] - data['Consumption'].shift(96)
# data = data[96:]
# data


In [ ]:
difference  = train['consumption'] - train['consumption'].shift(96)
train['consumption'] = difference
difference  = test['consumption'] - test['consumption'].shift(96)
test['consumption'] = difference 
train = train.iloc[96:]
test = test.iloc[96:]
test

In [10]:
# normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_diff = MinMaxScaler()
scaler_diff.fit(train[['consumption']])
train[train.columns] = scaler.fit_transform(train[train.columns])
test[test.columns] = scaler.transform(test[test.columns])

# data

# Min-Max Normalization: range [-0.5,0.5]
# data['consumption'] = (data['consumption'] - data['consumption'].mean())/(data['consumption'].max()-data['consumption'].min())

# # Decimal Scaling 
# data['consumption'] = data['consumption']/10000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3493: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [ ]:
import pickle

with open('Scalar.pickle', 'wb') as file:
    pickle.dump(scaler_diff,file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# for col in data.columns:
#     data[col] = 2*(data[col]-data[col].min())/(data[col].max()-data[col].min())
# data

In [ ]:
# windowing the dataset 
def windowing(df,target,win_size=96,feature_win_size=96,lag=96):
    '''
    Generate windowing DataFrame for df
    
    arguments: 
        df: target pandas.DataFrame
        target: the name of predictor
        win_size: the size of windows to feed in models as orignal data
        feature_win_size: the size of windows to analyze, but the original data is not feed into models. feature_win_size > win_size. 
    
    return: the new dataframe after windowing.'''
    from numpy.fft import fft 
    
    def get_outliers_rows(S):
        compare = (S-S.mean()).abs() > 2*S.std()
        windows_stat['outlier_max'] = S[compare].max()
        windows_stat['outlier_min'] = S[compare].min()
        windows_stat['outlier_distance'] = windows_stat['outlier_max'] - windows_stat['outlier_min']
        windows_stat['outlier_numbers'] = compare.sum()
        windows_stat.fillna(0,inplace=True)
        
    def entropy(series):
        import scipy.stats
        p_data = series.value_counts()
        entropy = scipy.stats.entropy(p_data)
        return entropy
    
    def negative_turning(series):
        point_num = 0
        last = 0
        for i,value in series.iteritems():
            if i==0:
                last = value
                continue 
            if value < last:
                point_num += 1 
            last = value 
        return point_num 
    
    def positive_turning(series):
        point_num = 0
        last = 0
        for i,value in series.iteritems():
            if i==0:
                last = value
                continue 
            if value > last:
                point_num += 1 
            last = value 
        return point_num 
        
    def get_slope(signal):
        """
        copy from tsfel 
        Computes the slope of the signal.
        Slope is computed by fitting a linear equation to the observed data.
        Feature computational cost: 1
        Parameters
        ----------
        signal : nd-array
            Input from which linear equation is computed
        Returns
        -------
        float
            Slope
        """
        t = np.linspace(0, len(signal) - 1, len(signal))
        return np.polyfit(t, signal, 1)[0]
    
    windows = pd.DataFrame()
    for i in range(0,feature_win_size):
        windows[str(i)] = df[target].shift(lag+i)
    windows = windows[lag+feature_win_size-1:] # Remove rows with NaN 
    
    windows_stat = pd.DataFrame()
    windows_stat['Mean'] = windows.mean(axis=1)
    windows_stat['Median'] = windows.median(axis=1)
    windows_stat['std'] = windows.std(axis=1)
#     windows.apply(get_outliers_rows,axis=1)
    windows_stat['skewness'] = windows.skew(axis=1)
    windows_stat['kurtosis'] = windows.kurtosis(axis=1)
    windows_stat['entropy'] = windows.apply(entropy,axis=1)
    windows_stat['25quantile'] = windows.quantile(0.25,axis=1)
    windows_stat['75quantile'] = windows.quantile(0.75,axis=1)
    windows_stat['InterquartileRange'] = windows_stat['75quantile'] - windows_stat['25quantile']
    windows_stat['mad'] = windows.mad(axis=1)
    windows_stat['RMS'] = windows_stat['Mean']**(1/2)
    windows_stat['Var'] = windows.var(axis=1)
#     windows_stat['NegativeTurning'] = windows.apply(negative_turning, axis=1)
#     windows_stat['PositiveTurning'] = windows.apply(negative_turning, axis=1)
    windows_stat['mean_abs_diff'] = windows.apply(lambda x: np.mean(np.abs(np.diff(x))),axis=1)
    windows_stat['mean_diff'] = windows.apply(lambda x: np.mean(np.diff(x)),axis=1)
    windows_stat['median_abs_diff'] = windows.apply(lambda x: np.median(np.abs(np.diff(x))),axis=1)
    windows_stat['sum_abs_diff'] = windows.apply(lambda x: np.sum(np.abs(np.diff(x))),axis=1)
    windows_stat['abs_energy'] = windows.apply(lambda x: np.sum(np.array(x) ** 2),axis=1)
    windows_stat['slope'] = windows.apply(get_slope,axis=1)
    windows_stat.fillna(0,inplace=True)
    
    windows_stat['96'] = df[target].shift(96)
    windows_stat['96'].fillna(windows_stat['96'].mean(),inplace=True)
    windows_stat['192'] = df[target].shift(192)
    windows_stat['192'].fillna(windows_stat['192'].mean(),inplace=True)
    windows_stat['288'] = df[target].shift(288)
    windows_stat['288'].fillna(windows_stat['288'].mean(),inplace=True)
    windows_stat['momentum'] = pd.DataFrame(windows_stat['96'] - windows_stat['192'])
    windows_stat['force'] = pd.DataFrame(windows_stat['96']- 2*windows_stat['192']+windows_stat['288'])
    windows_stat.drop(['96','192','288'],axis=1,inplace=True)
    
    
    windows_origin = pd.DataFrame()
    for i in range(0,win_size):
        windows_origin[str(i)] = df[target].shift(lag+i)
    windows_origin = windows_origin[lag+feature_win_size-1:] # Remove rows with NaN 

    return pd.concat([df[lag+feature_win_size-1:],windows_origin,windows_stat],axis=1)

In [ ]:
train = windowing(train,target='consumption')
test = windowing(test,target='consumption')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(train.dtypes)

In [ ]:
train.shape

In [ ]:
train.isnull().sum().sum()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(w_data.corr())

In [ ]:
f = plt.figure(figsize=(9,9))
sb.heatmap(w_data.corr())

In [ ]:
w_data.describe()

In [ ]:
train.to_csv('TrainData.csv')

In [ ]:
test.to_csv('TestData.csv')

In [ ]:
print(list(w_data.columns))

差值图

In [ ]:
difference  = data['consumption'] - data['consumption'].shift(96)

In [ ]:
f = plt.figure(figsize=(25,5))
sb.lineplot(x=data.index,y=data['consumption'])

In [ ]:
differnce2 = data['consumption'] - 2*data['consumption'].shift(96) + data['consumption'].shift(96*2)
f = plt.figure(figsize=(25,5))
sb.lineplot(x=differnce2.index,y=differnce2)